In [23]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [37]:
train_set = datasets.CIFAR10(root='./data', train=True, transform=transforms, download=True)
test_set = datasets.CIFAR10(root='./data', train=False, transform=transforms, download=True)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
train_set

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [29]:
# 定义反归一化函数
def denormalize(image):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = image * std + mean  # 反归一化
    return image.clamp(0, 1)  # 确保像素值在[0, 1]范围内

In [55]:
from PIL import Image
# CIFAR-10 类别名称
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

# 获取一张图片
img, target = train_set[0]  # 直接从 train_set 获取第一张图片
img = denormalize(img)  # 反归一化
img = img.permute(1, 2, 0).numpy()  # 转换为 [H, W, C] 格式并转为 numpy 数组
img = (img * 255).astype('uint8')  # 转换为 [0, 255] 的 uint8 类型，适配 PIL

# 使用 PIL 显示图片
# pil_img = Image.fromarray(img)
# pil_img.show(title=classes[target])  # 显示图片，标题为类别名称

from matplotlib import pyplot as plt
plt.figure(figsize=(4, 4), dpi=8)
plt.imshow(img)
plt.axis('off')
plt.show()
print(classes[target])

frog


In [57]:
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# 下载已经预训练好的模型 -- 这是个1000分类任务,在ImageNet的超大数据集上
model = models.resnet18(pretrained=True)

In [71]:
## 迁移学习
## 预训练模型的权重已经在 ImageNet（一个包含 1000 个类别的庞大数据集）上训练过，
## 模型已经学会了提取图像的通用特征（如边缘、纹理等）。
## 这使得它在其他任务上（如 CIFAR-10 的 10 类分类）有很好的起点。
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
device = torch.device('mps')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === 冻结除 model.fc 外的所有层 ===
for param in model.parameters():
    param.requires_grad = False  # 冻结所有参数
for param in model.fc.parameters():
    param.requires_grad = True   # 解冻 fc 层的参数
# ===================================

for epoch in range(10):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/100, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/100, Loss: 1.0213
Epoch 2/100, Loss: 0.9813
Epoch 3/100, Loss: 0.9773
Epoch 4/100, Loss: 0.9738
Epoch 5/100, Loss: 0.9665
Epoch 6/100, Loss: 0.9684
Epoch 7/100, Loss: 0.9624
Epoch 8/100, Loss: 0.9594
Epoch 9/100, Loss: 0.9530
Epoch 10/100, Loss: 0.9604


In [74]:
train_set[0][0].shape

torch.Size([3, 32, 32])

In [108]:
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)
model.train()
for epoch in range(20):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        images, labels = images.to(device), labels.to(device)
        pred = model(images)
        loss = criterion(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch: {epoch+1}/10, loss: {running_loss/len(train_loader):.4f}")

Files already downloaded and verified
Files already downloaded and verified
epoch: 1/10, loss: 1.5296
epoch: 2/10, loss: 1.2671
epoch: 3/10, loss: 1.1634
epoch: 4/10, loss: 1.1098
epoch: 5/10, loss: 1.0619
epoch: 6/10, loss: 1.0308
epoch: 7/10, loss: 1.0011
epoch: 8/10, loss: 0.9769
epoch: 9/10, loss: 0.9526
epoch: 10/10, loss: 0.9420
epoch: 11/10, loss: 0.9256
epoch: 12/10, loss: 0.9087
epoch: 13/10, loss: 0.8883
epoch: 14/10, loss: 0.8810
epoch: 15/10, loss: 0.8672
epoch: 16/10, loss: 0.8530
epoch: 17/10, loss: 0.8499
epoch: 18/10, loss: 0.8469
epoch: 19/10, loss: 0.8345
epoch: 20/10, loss: 0.8417


In [110]:
model.eval()
accuracy = 0
total = 0
print_cnt = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        y_pred = torch.argmax(outputs, dim=1)
        total += len(labels)
        cnt = torch.sum(y_pred == labels)
        accuracy += cnt.cpu().item()
        print_cnt += 1
        if print_cnt % 100 == 0:
            print(f"Accuracy: {accuracy/total:.4f}")

Accuracy: 0.6200
Accuracy: 0.6188
Accuracy: 0.6183
Accuracy: 0.6131
Accuracy: 0.6105
Accuracy: 0.6108
Accuracy: 0.6129
Accuracy: 0.6147
Accuracy: 0.6147
Accuracy: 0.6192
Accuracy: 0.6236
Accuracy: 0.6248
Accuracy: 0.6242
Accuracy: 0.6238
Accuracy: 0.6230
Accuracy: 0.6234
Accuracy: 0.6228
Accuracy: 0.6201
Accuracy: 0.6216
Accuracy: 0.6201
Accuracy: 0.6189
Accuracy: 0.6181
Accuracy: 0.6203
Accuracy: 0.6203
Accuracy: 0.6190


In [112]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with (torch.no_grad()):
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print(f'Accuracy of {classes[i]:5s}: {100 * class_correct[i] / class_total[i]:.2f}%')

Accuracy of airplane: 56.80%
Accuracy of automobile: 70.80%
Accuracy of bird : 57.90%
Accuracy of cat  : 53.40%
Accuracy of deer : 48.90%
Accuracy of dog  : 36.70%
Accuracy of frog : 74.40%
Accuracy of horse: 69.80%
Accuracy of ship : 77.10%
Accuracy of truck: 73.20%


In [114]:
# 保存模型参数
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [117]:
# 加载模型
net = Net()
net.load_state_dict(torch.load(PATH))

# 使用模型进行预测
with (torch.no_grad()):
    for images, labels in test_loader:
        # images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

Predicted:  cat   airplane ship  ship 
Predicted:  frog  frog  cat   frog 
Predicted:  cat   automobile ship  truck
Predicted:  deer  horse truck ship 
Predicted:  cat   cat   ship  frog 
Predicted:  horse bird  deer  truck
Predicted:  deer  deer  cat   bird 
Predicted:  truck frog  cat   cat  
Predicted:  bird  cat   truck cat  
Predicted:  deer  automobile truck dog  
Predicted:  airplane frog  horse frog 
Predicted:  ship  truck cat   ship 
Predicted:  horse bird  bird  ship 
Predicted:  frog  cat   ship  ship 
Predicted:  horse cat   bird  cat  
Predicted:  horse dog   airplane cat  
Predicted:  frog  frog  automobile bird 
Predicted:  horse horse bird  frog 
Predicted:  ship  ship  truck bird 
Predicted:  airplane cat   dog   ship 
Predicted:  ship  automobile automobile horse
Predicted:  bird  horse airplane deer 
Predicted:  ship  truck airplane deer 
Predicted:  ship  frog  horse cat  
Predicted:  frog  deer  airplane truck
Predicted:  deer  cat   frog  cat  
Predicted:  automo